<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Predicting Shots Made Per Game by Kobe Bryant

_Authors: Kiefer Katovich (SF)_

---

In this lab you'll be using regularized regression penalties — ridge, lasso, and elastic net — to try and predict how many shots Kobe Bryant made per game during his career.

The Kobe Shots data set contains hundreds of columns representing different characteristics of each basketball game. Fitting an ordinary linear regression using every predictor would dramatically overfit the model, considering the limited number of observations (games) we have available. Plus, many of the predictors have significant multicollinearity. 


**Warning:** Some of these calculations are computationally expensive and may take a while to execute. It may be worthwhile to only use a portion of the data to perform these calculations, especially if you've experienced kernel issues in the past.

---

### 1) Load packages and data.

In [1]:
import numpy as np
import pandas as pd
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
kobe = pd.read_csv('./datasets/kobe_superwide_games.csv')

---

### 2) Examine the data.

- How many columns are there?
- Examine what the observations (rows) and columns represent.
- Why might regularization be particularly useful for modeling this data?

In [3]:
kobe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1558 entries, 0 to 1557
Columns: 645 entries, SHOTS_MADE to CAREER_GAME_NUMBER
dtypes: float64(640), int64(5)
memory usage: 7.7 MB


In [8]:
kobe.head(3)

,SHOTS_MADE,AWAY_GAME,SEASON_OPPONENT:atl:1996-97,SEASON_OPPONENT:atl:1997-98,SEASON_OPPONENT:atl:1999-00,SEASON_OPPONENT:atl:2000-01,SEASON_OPPONENT:atl:2001-02,SEASON_OPPONENT:atl:2002-03,SEASON_OPPONENT:atl:2003-04,SEASON_OPPONENT:atl:2004-05,...,ACTION_TYPE:tip_layup_shot,ACTION_TYPE:tip_shot,ACTION_TYPE:turnaround_bank_shot,ACTION_TYPE:turnaround_fadeaway_bank_jump_shot,ACTION_TYPE:turnaround_fadeaway_shot,ACTION_TYPE:turnaround_finger_roll_shot,ACTION_TYPE:turnaround_hook_shot,ACTION_TYPE:turnaround_jump_shot,SEASON_GAME_NUMBER,CAREER_GAME_NUMBER
0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
2,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3


* There are 645 columns, since there are too many features which could result extremly overfitting, so regularzation is a good choice

---

### 3) Create predictor and target variables. Standardize the predictors.

Why is normalization necessary for regularized regressions?

Use the `sklearn.preprocessing` class `StandardScaler` to standardize the predictors.

In [4]:
X = kobe.drop('SHOTS_MADE',axis=1)
y = kobe['SHOTS_MADE']

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_standard = sc.fit_transform(X)


---

### 4. Build a linear regression predicting `SHOTS_MADE` from the rest of the columns.

Cross-validate the $R^2$ of an ordinary linear regression model with 10 cross-validation folds.

How does it perform?

In [12]:
linreg = LinearRegression()

cross_val_score(linreg,X_standard,y,cv=10)


array([-1.05853863e+29, -3.50275496e+28, -1.62805206e+28, -7.10909776e+27,
       -8.33950090e+27, -4.88812121e+27, -2.15167127e+27, -3.31369168e+27,
       -5.52897934e+27, -3.87976278e+28])

### The R2 is very large.

---

### 5) Find an optimal value for the ridge regression alpha using `RidgeCV`.

Go to the documentation and [read how RidgeCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).

> *Hint: Once the RidgeCV is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that ridge performs best when searching alphas through logarithmic space (`np.logspace`). This may take awhile to fit.


In [52]:
rg = RidgeCV(alphas=np.logspace(2,3,30),cv=5)
rg.fit(X_standard,y)

RidgeCV(alphas=array([ 100.        ,  108.26367339,  117.21022975,  126.89610032,
        137.38237959,  148.73521073,  161.02620276,  174.33288222,
        188.73918221,  204.33597179,  221.22162911,  239.502662  ,
        259.29437974,  280.72162039,  303.91953823,  329.03445623,
        356.22478903,  385.66204212,  417.53189366,  452.03536564,
        489.39009185,  529.83169063,  573.61525104,  621.01694189,
        672.33575365,  727.8953844 ,  788.04628157,  853.16785242,
        923.67085719, 1000.        ]),
        cv=5)

In [53]:
rg.alpha_

573.6152510448682

---

### 6) Cross-validate the ridge regression $R^2$ with the optimal alpha.

Is it better than the linear regression? If so, why might this be?

In [54]:
rg = RidgeCV(alphas=[573.62],cv=5)
rg.fit(X_standard,y)
rg.score(X_standard,y)

0.787230378875301

## It's much better because of the regularzation

---

### 7) Find an optimal value for lasso regression alpha using `LassoCV`.

Go to the documentation and [read how LassoCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html). It is very similar to `RidgeCV`.

> *Hint: Again, once the `LassoCV` is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that lasso, unlike ridge, performs best when searching for alpha through linear space (`np.linspace`). However, you can actually let the LassoCV decide what alphas to use itself by setting the keyword argument `n_alphas=` to however many alphas you want it to search over. We recommend letting scikit-learn choose the range of alphas.

_**Tip:** If you find your CV taking a long time and you're not sure if it's working, set `verbose =1`._

In [63]:
ls = LassoCV(n_alphas=10)
ls.fit(X_standard,y)
ls.score(X_standard,y)

/Users/czc/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:528: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.192621888132635, tolerance: 1.3344864365971105
  tol, rng, random, positive)
/Users/czc/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:528: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.6185722138707206, tolerance: 1.5443969526864476
  tol, rng, random, positive)


0.7267074628310961

---

### 8) Cross-validate the lasso $R^2$ with the optimal alpha.

Is it better than the linear regression? Is it better than ridge? What do the differences in results imply about the issues with the data set?

In [ ]:
# It's better than linear regression, but not as good as Ridge.
# Since Ridge and Lasso are both better the linear regression, which means the linear regression does make the model overfitting.

---

### 9) Look at the coefficients for variables in the lasso.

1. Show the coefficient for variables, ordered from largest to smallest coefficient by absolute value.
2. What percent of the variables in the original data set are "zeroed-out" by the lasso?
3. What are the most important predictors for how many shots Kobe made in a game?

> **Note:** If you only fit the lasso within `cross_val_score`, you'll have to refit it outside of that function to pull out the coefficients.

In [104]:
rg_coef = pd.DataFrame(index = X.columns,data = rg.coef_).reset_index()
rg_coef.columns = ['predictor','coef']
rg_coef['abs_coef'] = np.absolute(rg_coef['coef'])
rg_coef.sort_values(by='coef',ascending=False)

,predictor,coef,abs_coef
643,CAREER_GAME_NUMBER,0.280249,0.280249
642,SEASON_GAME_NUMBER,0.265963,0.265963
641,ACTION_TYPE:turnaround_jump_shot,0.263015,0.263015
640,ACTION_TYPE:turnaround_hook_shot,0.239537,0.239537
639,ACTION_TYPE:turnaround_finger_roll_shot,0.235186,0.235186
...,...,...,...
4,SEASON_OPPONENT:atl:2000-01,-0.097222,0.097222
3,SEASON_OPPONENT:atl:1999-00,-0.103091,0.103091
2,SEASON_OPPONENT:atl:1997-98,-0.109996,0.109996
1,SEASON_OPPONENT:atl:1996-97,-0.112607,0.112607


In [103]:
ls_coef = pd.DataFrame(index = X.columns,data = ls.coef_).reset_index()
ls_coef.columns = ['predictor','coef']
ls_coef['abs_coef'] = np.absolute(rg_coef['coef'])
ls_coef.sort_values(by='coef',ascending=False)

,predictor,coef,abs_coef
579,COMBINED_SHOT_TYPE:jump_shot,1.267017,1.267017
574,SHOT_TYPE:2pt_field_goal,0.828362,0.828362
566,SHOT_ZONE_BASIC:restricted_area,0.469498,0.469498
577,COMBINED_SHOT_TYPE:dunk,0.278567,0.278567
423,SEASON_OPPONENT:sac:1999-00,0.140978,0.140978
...,...,...,...
38,SEASON_OPPONENT:bos:2015-16,-0.100260,0.100260
594,ACTION_TYPE:driving_floating_jump_shot,-0.103651,0.103651
549,SEASON:2015-16,-0.125497,0.125497
612,ACTION_TYPE:layup_shot,-0.140674,0.140674


In [111]:
ls_coef[ls_coef['abs_coef'] == 0].shape[0]

474

In [112]:
print('There are 474 variables\' are 0 in Lasso, the most import variables are different in Ridge and Lasso')

There are 474 variables' are 0 in Lasso, the most import variables are different in Ridge and Lasso


---

### 10) Find an optimal value for elastic net regression alpha using `ElasticNetCV`.

Go to the documentation and [read how ElasticNetCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html).

Note that here you'll be optimizing both the alpha parameter and the l1_ratio:
- `alpha`: Strength of regularization.
- `l1_ratio`: Amount of ridge vs. lasso (0 = all ridge, 1 = all lasso).
    
Do not include 0 in the search for `l1_ratio` — it won't allow it and will break.

You can use `n_alphas` for the alpha parameters instead of setting your own values, which we highly recommend.

Also, be careful setting too many l1_ratios over cross-validation folds in your search. It can take a long time if you choose too many combinations and, for the most part, there are diminishing returns in this data.

In [124]:
ela = ElasticNetCV(n_alphas=10,l1_ratio=[0.1,0.3,0.5,0.7,0.9])
ela.fit(X_standard,y)

/Users/czc/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:528: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.7972424111890177, tolerance: 1.5554606741573034
  tol, rng, random, positive)


ElasticNetCV(l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9], n_alphas=10)

In [121]:
ela.alpha_

0.08078187932950469

In [125]:
ela.l1_ratio_

0.9

---

### 11) Cross-validate the elastic net $R^2$ with the optimal alpha and l1_ratio.

How does it compare to the ridge and lasso regularized regressions?

In [126]:
ela_new = ElasticNet(alpha=0.08078,l1_ratio=0.9)
cross_val_score(ela_new,X_standard,y,cv=5)

array([0.69156061, 0.5874929 , 0.55326052, 0.48639825, 0.53984163])

---

### 12. [Bonus] Compare the residuals for ridge and lasso visually.


In [ ]:
# A: Maybe a jointplot?